In [ ]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#wrapper for continuous environment
def ppo_eval(p, L, t_t, n_iter, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_config_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, learning_rate = learning_rate,use_sde = False)
    cont_model.learn(total_timesteps=t_t)#testing 2000

    cont_model.save("inv_cont_2")
    trained_model = PPO.load("inv_cont_2")
    env_eval = make_vec_env('inventory_cont_config_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    numiter = n_iter#test
    res_mean, res_std = ppo_evaluate(trained_model, env_eval, numiter)
    print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

    return res_mean, res_std

In [ ]:
# listp = [0.25,1,4,9,39,99]
# listL = [1,4,10,20,30,50,70,100]
listp = [39]
listL = [1]
t_t = 200000
n_iter = 100000
learning_rate = 0.005
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ppo_eval(p,L,t_t, n_iter, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':res_mean, 'res_std': res_std}, ignore_index=True)

In [ ]:
ppo_res

In [ ]:
#wrapper for cont env, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_config_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, learning_rate = learning_rate,use_sde = False)
    env_eval = make_vec_env('inventory_cont_config_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=8000)#testing 2000
        timesteps = timesteps + 8192

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(res_mean)
        res_std_arr.append(res_std)
    
    plot(res_mean_arr)

    return res_mean_arr, res_std_arr

In [ ]:
# listp = [0.25,1,4,9,39,99]
# listL = [1,4,10,20,30,50,70,100]
listp = [99]
listL = [100]
t_t = 2000000
n_iter = 1000
learning_rate = 0.003
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ppo_eval_interval(p,L,t_t, n_iter, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':res_mean, 'res_std': res_std}, ignore_index=True)

In [ ]:
#continuous model try with cont action

# #env = make_vec_env('inventory_cont_model-v0', ContCONFIG = {'h': 1, 'p': 1, 'L': 10, 'lambda': 1}, n_envs=4)
# #how to set parameters???
# cont_env = make_vec_env('inventory_cont_model-v0', n_envs=8)
# cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
# cont_model.learn(total_timesteps=200000)
# # fixed issue with vector, now action can be continuous

# #continuous evaluation
# cont_model.save("inv_cont_1")
# trained_model = PPO.load("inv_cont_1")
# env1 = gym.make('inventory_cont_model-v0')
# numiter = 50000
# res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
# print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#Discrete environment
#h=1,p=99,L=1,max_inventory=100,max_action=100,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_2")

trained_model2 = PPO.load("inv_2")
env2 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model2, env2, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#Discrete environment 2
#h=1,p=99,L=1,max_inventory=100,max_action=6,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_3")

#set action space to be small([0,1,2,3,4,5,6]) seems to get reasonable results...
trained_model3 = PPO.load("inv_3")
env3 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model3, env3, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 